# Quickstart

In this quickstart you will create a simple Llama Index App and learn how to log it and get feedback on an LLM response.

## Setup

### Install dependencies
Let's install some of the dependencies for this notebook if we don't have them already

In [1]:
%load_ext autoreload

In [2]:
#!pip install trulens-eval
#!pip install llama_index==0.6.31

### Add API keys
For this quickstart, you will need Open AI and Huggingface keys

In [3]:
#import os
#os.environ["OPENAI_API_KEY"] = "..."
#os.environ["HUGGINGFACE_API_KEY"] = "..."

### Import from LlamaIndex and TruLens

In [4]:
# Imports main tools:
from trulens_eval import TruLlama, Feedback, Tru, feedback
tru = Tru()


No .env found in /home/shayak/code/trulens/trulens_eval/examples/frameworks/llama_index or its parents. You may need to specify secret keys in another manner.


### Create Simple LLM Application

This example uses LlamaIndex which internally uses an OpenAI LLM.

In [5]:
# LLama Index starter example from: https://gpt-index.readthedocs.io/en/latest/getting_started/starter_example.html
# In order to run this, download into data/ Paul Graham's Essay 'What I Worked On' from https://github.com/jerryjliu/llama_index/blob/main/examples/paul_graham_essay/data/paul_graham_essay.txt 

from llama_index import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader('data').load_data()
index = VectorStoreIndex.from_documents(documents)

query_engine = index.as_query_engine()

### Send your first request

In [6]:
#response = query_engine.query("What did the author do growing up?")
#print(response)

## Initialize Feedback Function(s)

In [7]:
import numpy as np

# Initialize Huggingface-based feedback function collection class:
hugs = feedback.Huggingface()
openai = feedback.OpenAI()

# Define a language match feedback function using HuggingFace.
f_lang_match = Feedback(hugs.language_match).on_input_output()
# By default this will check language match on the main app input and main app
# output.

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(openai.relevance).on_input_output()

# Question/statement relevance between question and each context chunk.
f_qs_relevance = Feedback(openai.qs_relevance).on_input().on(
    TruLlama.select_source_nodes().node.text
).aggregate(np.min)

✅ In language_match, input text1 will be set to *.__record__.main_input or `Select.RecordInput` .
✅ In language_match, input text2 will be set to *.__record__.main_output or `Select.RecordOutput` .
✅ In relevance, input prompt will be set to *.__record__.main_input or `Select.RecordInput` .
✅ In relevance, input response will be set to *.__record__.main_output or `Select.RecordOutput` .
✅ In qs_relevance, input question will be set to *.__record__.main_input or `Select.RecordInput` .
✅ In qs_relevance, input statement will be set to *.__record__.app.query.rets.source_nodes[:].node.text .


In [8]:
golden_set = [
    {"query": "What was the author's undergraduate major?", "response": "He didn't choose a major, and customized his courses."},
    {"query": "What company did the author start in 1995?", "response": "Viaweb, to make software for building online stores."},
    {"query": "Where did the author move in 1998 after selling Viaweb?", "response": "California, after Yahoo acquired Viaweb."},
    {"query": "What did the author do after leaving Yahoo in 1999?", "response": "He focused on painting and tried to improve his art skills."},
    {"query": "What program did the author start with Jessica Livingston in 2005?", "response": "Y Combinator, to provide seed funding for startups."}
]



In [9]:
f_groundtruth = Feedback(openai.GroundTruthAgreement(golden_set).measure).on_input_output()

✅ In measure, input prompt will be set to *.__record__.main_input or `Select.RecordInput` .
✅ In measure, input response will be set to *.__record__.main_output or `Select.RecordOutput` .


## Instrument chain for logging with TruLens

In [10]:
tru_query_engine = TruLlama(query_engine,
    app_id='LlamaIndex_App1',
    feedbacks=[f_lang_match, f_qa_relevance, f_qs_relevance, f_groundtruth])

✅ app LlamaIndex_App1 -> default.sqlite
✅ feedback def. feedback_definition_hash_396bb835ef1fca2f0433f55c6677818f -> default.sqlite
✅ feedback def. feedback_definition_hash_97445f1de6f182f1ebeb2e8927251586 -> default.sqlite
✅ feedback def. feedback_definition_hash_62739b7af9cbe724692d5fbac610eed1 -> default.sqlite
✅ feedback def. feedback_definition_hash_010ec7470e39c927ca7574958baf9009 -> default.sqlite


In [13]:
# Instrumented query engine can operate like the original:
llm_response = tru_query_engine.query("Where was the author born?")

print(llm_response)


The author does not mention where they were born in the context information provided.
✅ record record_hash_6ee85b5caa67db08218d60cb63070aad from LlamaIndex_App1 -> default.sqlite
[{'query': "What was the author's undergraduate major?", 'response': "He didn't choose a major, and customized his courses."}, {'query': 'What company did the author start in 1995?', 'response': 'Viaweb, to make software for building online stores.'}, {'query': 'Where did the author move in 1998 after selling Viaweb?', 'response': 'California, after Yahoo acquired Viaweb.'}, {'query': 'What did the author do after leaving Yahoo in 1999?', 'response': 'He focused on painting and tried to improve his art skills.'}, {'query': 'What program did the author start with Jessica Livingston in 2005?', 'response': 'Y Combinator, to provide seed funding for startups.'}]
[]
Where was the author born?
None


Waiting for {'error': 'Model papluca/xlm-roberta-base-language-detection is currently loading', 'estimated_time': 44.49275207519531} (44.49275207519531) second(s).
Waiting for {'error': 'Model papluca/xlm-roberta-base-language-detection is currently loading', 'estimated_time': 44.49275207519531} (44.49275207519531) second(s).


✅ feedback feedback_result_hash_b35792366a28414504ef0c84edc9e5d4 on record_hash_6ee85b5caa67db08218d60cb63070aad -> default.sqlite
✅ feedback feedback_result_hash_cc93ad22768d40f16827e1388e2afe99 on record_hash_6ee85b5caa67db08218d60cb63070aad -> default.sqlite
✅ feedback feedback_result_hash_dbbc15fbefca1d9c7b6285ad7ba9e821 on record_hash_6ee85b5caa67db08218d60cb63070aad -> default.sqlite
✅ feedback feedback_result_hash_e6b457c10ab9badae2eb5c7817ae5c4a on record_hash_6ee85b5caa67db08218d60cb63070aad -> default.sqlite


## Explore in a Dashboard

In [12]:
tru.run_dashboard() # open a local streamlit app to explore

# tru.stop_dashboard() # stop if needed

Starting dashboard ...


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://172.31.253.65:8502 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

### Leaderboard

Understand how your LLM application is performing at a glance. Once you've set up logging and evaluation in your application, you can view key performance statistics including cost and average feedback value across all of your LLM apps using the chain leaderboard. As you iterate new versions of your LLM application, you can compare their performance across all of the different quality metrics you've set up.

Note: Average feedback values are returned and displayed in a range from 0 (worst) to 1 (best).

![Chain Leaderboard](https://www.trulens.org/Assets/image/Leaderboard.png)

To dive deeper on a particular chain, click "Select Chain".

### Understand chain performance with Evaluations
 
To learn more about the performance of a particular chain or LLM model, we can select it to view its evaluations at the record level. LLM quality is assessed through the use of feedback functions. Feedback functions are extensible methods for determining the quality of LLM responses and can be applied to any downstream LLM task. Out of the box we provide a number of feedback functions for assessing model agreement, sentiment, relevance and more.

The evaluations tab provides record-level metadata and feedback on the quality of your LLM application.

![Evaluations](https://www.trulens.org/Assets/image/Leaderboard.png)

### Deep dive into full chain metadata

Click on a record to dive deep into all of the details of your chain stack and underlying LLM, captured by tru_chain.

![Explore a Chain](https://www.trulens.org/Assets/image/Chain_Explore.png)

If you prefer the raw format, you can quickly get it using the "Display full chain json" or "Display full record json" buttons at the bottom of the page.

Note: Feedback functions evaluated in the deferred manner can be seen in the "Progress" page of the TruLens dashboard.

## Or view results directly in your notebook

tru.get_records_and_feedback(app_ids=[])[0] # pass an empty list of app_ids to get all